In [1]:
import os
import re
import time
import nltk
import re
import string
import tensorlayer as tl
from utils import *

/home/ye/user/yejg/SW_DATA/Or/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
dataset='102flowers'
need_256=True

In [4]:
cwd=os.getcwd()
img_dir=os.path.join(cwd,dataset)
caption_dir=os.path.join(cwd,'cvpr2016_flowers/','text_c10')
VOC_FIR=cwd+'/vocab.txt'

###  load captions

In [6]:
caption_sub_dir=load_folder_list(caption_dir)
caption_dict={}
processed_capts=[]
print(caption_dir,'\n',caption_sub_dir[1:5])

/home/ye/user/yejg/LEARN/DL_MODEL_LEARN/TensorFlow/GAN/stack-text-to-image/cvpr2016_flowers/text_c10 
 ['/home/ye/user/yejg/LEARN/DL_MODEL_LEARN/TensorFlow/GAN/stack-text-to-image/cvpr2016_flowers/text_c10/class_00026', '/home/ye/user/yejg/LEARN/DL_MODEL_LEARN/TensorFlow/GAN/stack-text-to-image/cvpr2016_flowers/text_c10/class_00039', '/home/ye/user/yejg/LEARN/DL_MODEL_LEARN/TensorFlow/GAN/stack-text-to-image/cvpr2016_flowers/text_c10/class_00049', '/home/ye/user/yejg/LEARN/DL_MODEL_LEARN/TensorFlow/GAN/stack-text-to-image/cvpr2016_flowers/text_c10/class_00083']


In [7]:
files=tl.files.load_file_list(path=caption_sub_dir[1],regx='^image_[0-9]+\.txt')

Match file list = ['image_06519.txt', 'image_06491.txt', 'image_06494.txt', 'image_06521.txt', 'image_06500.txt', 'image_06488.txt', 'image_06508.txt', 'image_06513.txt', 'image_06507.txt', 'image_06515.txt', 'image_06495.txt', 'image_06514.txt', 'image_06506.txt', 'image_06522.txt', 'image_06489.txt', 'image_06527.txt', 'image_06504.txt', 'image_06490.txt', 'image_06525.txt', 'image_06496.txt', 'image_06497.txt', 'image_06516.txt', 'image_06499.txt', 'image_06526.txt', 'image_06524.txt', 'image_06503.txt', 'image_06498.txt', 'image_06517.txt', 'image_06493.txt', 'image_06509.txt', 'image_06510.txt', 'image_06505.txt', 'image_06523.txt', 'image_06512.txt', 'image_06487.txt', 'image_06501.txt', 'image_06502.txt', 'image_06520.txt', 'image_06518.txt', 'image_06492.txt', 'image_06511.txt']
Number of files = 41


In [9]:
for sub_dir in caption_sub_dir:
    with tl.ops.suppress_stdout():
        files=tl.files.load_file_list(path=sub_dir,regx='^image_[0-9]+\.txt')
        for i,f in enumerate(files):
            file_dir=os.path.join(sub_dir,f)
            key=int(re.findall('\d+',f)[0])
            t=open(file_dir,'r')
            lines=[]
            for line in t:
                line=preprocess_caption(line)
                lines.append(line)
                processed_capts.append(tl.nlp.process_sentence(line,start_word='<S>',end_word='</S>'))
            assert len(lines)==10
            caption_dict[key]=lines
print(" * %d x %d captions found " % (len(caption_dict), len(lines)))

NameError: name 'captions_dict' is not defined

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os.path
import sys
import tarfile

import numpy as np
from six.moves import urllib
import tensorflow as tf
import glob
import scipy.misc
import math
import sys

MODEL_DIR = './imagenet'
DATA_URL = 'http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz'
softmax = None

/home/ye/user/yejg/SW_DATA/Or/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def get_inception_score(images, splits=10):
  assert(type(images) == list)
  assert(type(images[0]) == np.ndarray)
  assert(len(images[0].shape) == 3)
  assert(np.max(images[0]) > 10)
  assert(np.min(images[0]) >= 0.0)
  inps = []
  for img in images:
    img = img.astype(np.float32)
    inps.append(np.expand_dims(img, 0))
  bs = 100
  with tf.Session() as sess:
    preds = []
    n_batches = int(math.ceil(float(len(inps)) / float(bs)))
    for i in range(n_batches):
        sys.stdout.write(".")
        sys.stdout.flush()
        inp = inps[(i * bs):min((i + 1) * bs, len(inps))]
        inp = np.concatenate(inp, 0)
        pred = sess.run(softmax, {'ExpandDims:0': inp})
        preds.append(pred)
    preds = np.concatenate(preds, 0)
    scores = []
    for i in range(splits):
      part = preds[(i * preds.shape[0] // splits):((i + 1) * preds.shape[0] // splits), :]
      kl = part * (np.log(part) - np.log(np.expand_dims(np.mean(part, 0), 0)))
      kl = np.mean(np.sum(kl, 1))
      scores.append(np.exp(kl))
    return np.mean(scores), np.std(scores)

In [3]:
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)
filename = DATA_URL.split('/')[-1]
filepath = os.path.join(MODEL_DIR, filename)

with tf.gfile.FastGFile(os.path.join(
      MODEL_DIR, 'classify_image_graph_def.pb'), 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    _ = tf.import_graph_def(graph_def, name='')

In [13]:
# Works with an arbitrary minibatch size.
with tf.Session() as sess:
    pool3 = sess.graph.get_tensor_by_name('pool_3:0')
    ops = pool3.graph.get_operations()
    for op_idx, op in enumerate(ops):
        for o in op.outputs:
            shape = o.get_shape()
            shape = [s.value for s in shape]
            new_shape = []
            for j, s in enumerate(shape):
                if s == 1 and j == 0:
                    new_shape.append(None)
                else:
                    new_shape.append(s)
            #o._shape = tf.TensorShape(new_shape)
            o.set_shape=tf.TensorShape(new_shape)
    w = sess.graph.get_operation_by_name("softmax/logits/MatMul").inputs[1]
    print(tf.transpose(w))
    print(pool3)
    logits = tf.matmul(tf.transpose(w),tf.expand_dims(tf.squeeze(pool3),1))
    prob = tf.nn.softmax(logits)

Tensor("transpose_3:0", shape=(1008, 2048), dtype=float32)
Tensor("pool_3:0", shape=(1, 1, 1, 2048), dtype=float32)


In [9]:
print(tf.expand_dims(tf.squeeze(pool3),1))

Tensor("ExpandDims_1:0", shape=(2048, 1), dtype=float32)


In [12]:
tf.nn.softmax(logits)

NameError: name 'logits' is not defined